In [1]:
import sys
import os
from pathlib import Path

root = Path(os.getcwd()).parent.parent
sys.path.append(str(Path(os.getcwd()).parent))

In [2]:
from loader import Power, Weather
from constant import FeatureType
from keras.layers import LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.python.keras.optimizer_v2.rmsprop import RMSProp
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error,r2_score
from datetime import datetime, timedelta

import json
import argparse
import numpy as np
import pandas as pd
import tensorflow as tf

### Setting

In [3]:
parser = argparse.ArgumentParser()
args = parser.parse_args("")

# ====== Path ====== #
args.root = root

# ====== Model ====== #
args.frame_in = 72
args.frame_out = 24

# ====== Data ====== #
args.years = [2017, 2018, 2019]
args.region = "Jindo"
args.station = 192
args.ratio = [0.6, 0.2, 0.2]

# ====== Features ====== #
features = [FeatureType.SUNSHINE,
            FeatureType.GROUND_TEMPERATURE,
            FeatureType.HUMIDITY,
            FeatureType.WIND_SPEED,
            FeatureType.WIND_DIRECTION,
            FeatureType.TEMPERATURE,
            FeatureType.VISIBILITY,
            FeatureType.PRECIPITATION,
            FeatureType.STEAM_PRESSURE,
            FeatureType.DEW_POINT_TEMPERATURE,
            FeatureType.ATMOSPHERIC_PRESSURE]

### Saved settings

In [4]:
settings = []

result_path = os.path.join(args.root, 'results', 'automl')
for i in range(11):
#     filepath = os.path.join(result_path, 'model %d setting.txt' % (i + 1))
#     with open(filepath) as f:
#         json_data = json.load(f)
#     settings.append(json_data)
    settings.append({"batch_size": 64, "learning_rate": 0.001, "patience": 30, "shuffle": True})

### Model

In [5]:
def encoder_decoder(args, dataset, budget):
    print("train")

    X_train, y_train = dataset['train']
    X_val, y_val = dataset['val']
    X_test, y_test = dataset['test']

    with tf.device('/GPU:0'):
        model = Sequential()
        optimizer = RMSProp(learning_rate=args.learning_rate)

        model.add(LSTM(256, input_shape=(args.frame_in, args.feature_len)))
        model.add(RepeatVector(args.frame_out))
        model.add(LSTM(256, return_sequences=True))
        model.add(TimeDistributed(Dense(256, activation='relu')))
        model.add(TimeDistributed(Dense(1)))
        model.compile(loss='mse', optimizer=optimizer)

        model_root_path = os.path.join(args.root, 'models', args.name)
        Path(model_root_path).mkdir(parents=True, exist_ok=True)
        checkpoint_path = os.path.join(model_root_path, 'model-{epoch:03d}-{val_loss:03f}.h5')
        model_path = os.path.join(model_root_path, 'model.h5')

        callback = EarlyStopping(monitor='val_loss', patience=args.patience)
        checkpoint = ModelCheckpoint(checkpoint_path, verbose=1, monitor='val_loss', save_best_only=True)

        history = model.fit(X_train, y_train, batch_size=args.batch_size, epochs=budget,
                            validation_data=(X_val, y_val), callbacks=[callback, checkpoint], shuffle=args.shuffle)
        model.save(model_path)
        
        return model
    
def predict(partition, model, scaler):
    X_test, y_test = partition['test']
    y_test = y_test.reshape((y_test.shape[0] * y_test.shape[1], y_test.shape[2]))
    y_test = scaler.inverse_transform(y_test)
    
    y_pred = model.predict(X_test)
    y_pred = y_pred.reshape((-1, 1))
    y_pred = scaler.inverse_transform(y_pred)
    
    print(y_test.shape, y_pred.shape)
    
    zero_indices = np.where(y_test == 0)
    y_test = np.delete(y_test, zero_indices)
    y_pred = np.delete(y_pred, zero_indices)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    max_min = np.max(y_test) - np.min(y_test)
    nrmse = rmse / max_min
    
    print('rmse:', rmse)
    print('nrmse:', nrmse)
    
    return y_test, y_pred, nrmse

def save(y_pred_list, y_test, test_start, test_end):
    df = pd.DataFrame()
    for i in range(len(features)):
        y_pred_list[i] = y_pred_list[i].reshape((y_pred_list[i].shape[0]))
        df['%dth model' % (i + 1)] = y_pred_list[i].tolist()
    y_test = y_test.reshape((y_test.shape[0]))
    df['y_test'] = y_test.tolist()

    full_idx = pd.date_range(start=test_start+timedelta(days=3), end=test_end, freq='H')
    full_idx = full_idx[:y_test.shape[0]]
    df['time'] = full_idx
    df = df.set_index('time')

    result_path = os.path.join(root, 'results')
    result_name = os.path.join(result_path, 'result.csv')
    df.to_csv(result_name)

### Dataset

In [6]:
power = Power(args)
weather = Weather(args, features)

power_data = power.get_data()
scaler = power_data['scaler']

y_pred_list = []
nrmse_list = []

for i in range(len(features)):
    weather_data = weather.get_data(i+1)
    setattr(args, 'feature_len', i + 1)
    setattr(args, 'name', 'model %d' % (i + 1))
    setattr(args, 'batch_size', settings[i]['batch_size'])
    setattr(args, 'learning_rate', settings[i]['learning_rate'])
    setattr(args, 'patience', settings[i]['patience'])
    setattr(args, 'shuffle', settings[i]['shuffle'])

    train = [weather_data['train'], power_data['train']]
    val = [weather_data['val'], power_data['val']]
    test = [weather_data['test'], power_data['test']]

    dataset = {'train': train, 'val': val, 'test': test}
    
    model = encoder_decoder(args, dataset, budget=256)
    y_test, y_pred, nrmse = predict(dataset, model, scaler)
    y_pred_list.append(y_pred)
    nrmse_list.append(nrmse)
    
save(y_pred_list, y_test, power.test_start, power.test_end)

train start date: 2017-01-01 00:00:00
train end date: 2018-10-19 23:00:00
val start date: 2018-10-20 00:00:00
val end date: 2019-05-26 23:00:00
test start date: 2019-05-27 00:00:00
test end date: 2019-12-31 23:00:00
train start date: 2017-01-01 00:00:00
train end date: 2018-10-19 23:00:00
val start date: 2018-10-20 00:00:00
val end date: 2019-05-26 23:00:00
test start date: 2019-05-27 00:00:00
test end date: 2019-12-31 23:00:00
0 missing dates
0 value(s) are not zero
0 missing dates
0 value(s) are not zero
0 missing dates
0 value(s) are not zero
(15768,)
(5256,)
(5256,)
missing dates: []
missing dates: ['2018-01-13 09:00', '2018-01-13 10:00', '2018-01-13 11:00', '2018-01-13 12:00', '2018-01-13 13:00']
missing dates: []
train
Epoch 1/256
 7/11 [==================>...........] - ETA: 0s - loss: 0.0096
Epoch 00001: val_loss improved from inf to 0.00145, saving model to /home/vislab/Github/SolarPowerPrediction/models/model 1/model-001-0.001455.h5
11/11 [==============================] - 1s

 8/11 [====================>.........] - ETA: 0s - loss: 5.2501e-04
Epoch 00031: val_loss did not improve from 0.00030
11/11 [==============================] - 0s 10ms/step - loss: 5.0596e-04 - val_loss: 3.3628e-04
Epoch 32/256
 8/11 [====================>.........] - ETA: 0s - loss: 4.7109e-04
Epoch 00032: val_loss did not improve from 0.00030
11/11 [==============================] - 0s 10ms/step - loss: 4.5355e-04 - val_loss: 7.1490e-04
Epoch 33/256
 8/11 [====================>.........] - ETA: 0s - loss: 5.7593e-04
Epoch 00033: val_loss did not improve from 0.00030
11/11 [==============================] - 0s 10ms/step - loss: 5.7620e-04 - val_loss: 5.9508e-04
Epoch 34/256
 8/11 [====================>.........] - ETA: 0s - loss: 3.9100e-04
Epoch 00034: val_loss did not improve from 0.00030
11/11 [==============================] - 0s 10ms/step - loss: 4.5632e-04 - val_loss: 4.4577e-04
Epoch 35/256
 8/11 [====================>.........] - ETA: 0s - loss: 5.5608e-04
Epoch 00035: val_los

Epoch 6/256
 8/11 [====================>.........] - ETA: 0s - loss: 0.0014    
Epoch 00006: val_loss improved from 0.00056 to 0.00051, saving model to /home/vislab/Github/SolarPowerPrediction/models/model 2/model-006-0.000514.h5
11/11 [==============================] - 0s 12ms/step - loss: 0.0013 - val_loss: 5.1353e-04
Epoch 7/256
 8/11 [====================>.........] - ETA: 0s - loss: 7.3972e-04
Epoch 00007: val_loss did not improve from 0.00051
11/11 [==============================] - 0s 10ms/step - loss: 7.3634e-04 - val_loss: 6.0509e-04
Epoch 8/256
 8/11 [====================>.........] - ETA: 0s - loss: 6.9985e-04
Epoch 00008: val_loss improved from 0.00051 to 0.00047, saving model to /home/vislab/Github/SolarPowerPrediction/models/model 2/model-008-0.000472.h5
11/11 [==============================] - 0s 12ms/step - loss: 6.9903e-04 - val_loss: 4.7156e-04
Epoch 9/256
 8/11 [====================>.........] - ETA: 0s - loss: 6.5122e-04
Epoch 00009: val_loss did not improve from 0.

Epoch 40/256
 8/11 [====================>.........] - ETA: 0s - loss: 4.9404e-04
Epoch 00040: val_loss did not improve from 0.00030
11/11 [==============================] - 0s 10ms/step - loss: 4.6580e-04 - val_loss: 3.7083e-04
Epoch 41/256
 8/11 [====================>.........] - ETA: 0s - loss: 5.0673e-04
Epoch 00041: val_loss did not improve from 0.00030
11/11 [==============================] - 0s 10ms/step - loss: 4.8478e-04 - val_loss: 5.9044e-04
Epoch 42/256
 8/11 [====================>.........] - ETA: 0s - loss: 4.9972e-04
Epoch 00042: val_loss did not improve from 0.00030
11/11 [==============================] - 0s 10ms/step - loss: 4.6853e-04 - val_loss: 4.8153e-04
Epoch 43/256
 8/11 [====================>.........] - ETA: 0s - loss: 3.8640e-04
Epoch 00043: val_loss did not improve from 0.00030
11/11 [==============================] - 0s 10ms/step - loss: 4.4929e-04 - val_loss: 5.6308e-04
Epoch 44/256
 8/11 [====================>.........] - ETA: 0s - loss: 4.0159e-04
Epoch 0

(5184, 1) (5184, 1)
rmse: 66.34375061707446
nrmse: 0.21610342220545428
missing dates: []
missing dates: ['2018-01-13 09:00', '2018-01-13 10:00', '2018-01-13 11:00', '2018-01-13 12:00', '2018-01-13 13:00']
missing dates: []
train
Epoch 1/256
 8/11 [====================>.........] - ETA: 0s - loss: 0.0576
Epoch 00001: val_loss improved from inf to 0.00140, saving model to /home/vislab/Github/SolarPowerPrediction/models/model 3/model-001-0.001398.h5
11/11 [==============================] - 1s 48ms/step - loss: 0.0454 - val_loss: 0.0014
Epoch 2/256
 7/11 [==================>...........] - ETA: 0s - loss: 0.0016
Epoch 00002: val_loss did not improve from 0.00140
11/11 [==============================] - 0s 10ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 3/256
 8/11 [====================>.........] - ETA: 0s - loss: 0.0016
Epoch 00003: val_loss did not improve from 0.00140
11/11 [==============================] - 0s 10ms/step - loss: 0.0017 - val_loss: 0.0021
Epoch 4/256
 8/11 [============

...

 7/11 [==================>...........] - ETA: 0s - loss: 9.2265e-04
Epoch 00018: val_loss did not improve from 0.00068
11/11 [==============================] - 0s 10ms/step - loss: 7.8728e-04 - val_loss: 9.4908e-04
Epoch 19/256
 8/11 [====================>.........] - ETA: 0s - loss: 7.8867e-04
Epoch 00019: val_loss improved from 0.00068 to 0.00036, saving model to /home/vislab/Github/SolarPowerPrediction/models/model 10/model-019-0.000362.h5
11/11 [==============================] - 0s 12ms/step - loss: 7.6167e-04 - val_loss: 3.6223e-04
Epoch 20/256
 8/11 [====================>.........] - ETA: 0s - loss: 7.8190e-04
Epoch 00020: val_loss did not improve from 0.00036
11/11 [==============================] - 0s 10ms/step - loss: 7.1111e-04 - val_loss: 7.7811e-04
Epoch 21/256
 8/11 [====================>.........] - ETA: 0s - loss: 0.0011    
Epoch 00021: val_loss improved from 0.00036 to 0.00033, saving model to /home/vislab/Github/SolarPowerPrediction/models/model 10/model-021-0.000332.

Epoch 52/256
 8/11 [====================>.........] - ETA: 0s - loss: 4.1625e-04
Epoch 00052: val_loss did not improve from 0.00027
11/11 [==============================] - 0s 10ms/step - loss: 4.5595e-04 - val_loss: 3.1359e-04
Epoch 53/256
 8/11 [====================>.........] - ETA: 0s - loss: 3.9816e-04
Epoch 00053: val_loss improved from 0.00027 to 0.00027, saving model to /home/vislab/Github/SolarPowerPrediction/models/model 10/model-053-0.000268.h5
11/11 [==============================] - 0s 12ms/step - loss: 4.6529e-04 - val_loss: 2.6825e-04
Epoch 54/256
 8/11 [====================>.........] - ETA: 0s - loss: 3.8457e-04
Epoch 00054: val_loss did not improve from 0.00027
11/11 [==============================] - 0s 10ms/step - loss: 3.9941e-04 - val_loss: 3.1486e-04
Epoch 55/256
 8/11 [====================>.........] - ETA: 0s - loss: 4.7018e-04
Epoch 00055: val_loss did not improve from 0.00027
11/11 [==============================] - 0s 10ms/step - loss: 4.5560e-04 - val_loss:

Epoch 3/256
 7/11 [==================>...........] - ETA: 0s - loss: 0.0015
Epoch 00003: val_loss did not improve from 0.00134
11/11 [==============================] - 0s 10ms/step - loss: 0.0018 - val_loss: 0.0016
Epoch 4/256
 8/11 [====================>.........] - ETA: 0s - loss: 0.0015
Epoch 00004: val_loss did not improve from 0.00134
11/11 [==============================] - 0s 10ms/step - loss: 0.0016 - val_loss: 0.0021
Epoch 5/256
 8/11 [====================>.........] - ETA: 0s - loss: 0.0018
Epoch 00005: val_loss did not improve from 0.00134
11/11 [==============================] - 0s 10ms/step - loss: 0.0019 - val_loss: 0.0017
Epoch 6/256
 8/11 [====================>.........] - ETA: 0s - loss: 0.0019
Epoch 00006: val_loss did not improve from 0.00134
11/11 [==============================] - 0s 10ms/step - loss: 0.0018 - val_loss: 0.0013
Epoch 7/256
 8/11 [====================>.........] - ETA: 0s - loss: 0.0020
Epoch 00007: val_loss did not improve from 0.00134
11/11 [======

 8/11 [====================>.........] - ETA: 0s - loss: 4.4105e-04
Epoch 00036: val_loss improved from 0.00035 to 0.00033, saving model to /home/vislab/Github/SolarPowerPrediction/models/model 11/model-036-0.000329.h5
11/11 [==============================] - 0s 12ms/step - loss: 4.8972e-04 - val_loss: 3.2898e-04
Epoch 37/256
 7/11 [==================>...........] - ETA: 0s - loss: 5.3881e-04
Epoch 00037: val_loss did not improve from 0.00033
11/11 [==============================] - 0s 10ms/step - loss: 5.2179e-04 - val_loss: 5.4481e-04
Epoch 38/256
 8/11 [====================>.........] - ETA: 0s - loss: 4.2127e-04
Epoch 00038: val_loss did not improve from 0.00033
11/11 [==============================] - 0s 10ms/step - loss: 5.0151e-04 - val_loss: 6.3907e-04
Epoch 39/256
 8/11 [====================>.........] - ETA: 0s - loss: 5.4756e-04
Epoch 00039: val_loss improved from 0.00033 to 0.00032, saving model to /home/vislab/Github/SolarPowerPrediction/models/model 11/model-039-0.000320.

 8/11 [====================>.........] - ETA: 0s - loss: 3.4594e-04
Epoch 00070: val_loss did not improve from 0.00027
11/11 [==============================] - 0s 10ms/step - loss: 4.0803e-04 - val_loss: 2.9673e-04
Epoch 71/256
 8/11 [====================>.........] - ETA: 0s - loss: 3.5096e-04
Epoch 00071: val_loss did not improve from 0.00027
11/11 [==============================] - 0s 10ms/step - loss: 4.2259e-04 - val_loss: 4.6569e-04
Epoch 72/256
 8/11 [====================>.........] - ETA: 0s - loss: 4.2195e-04
Epoch 00072: val_loss did not improve from 0.00027
11/11 [==============================] - 0s 10ms/step - loss: 4.0283e-04 - val_loss: 2.9943e-04
Epoch 73/256
 8/11 [====================>.........] - ETA: 0s - loss: 3.2708e-04
Epoch 00073: val_loss did not improve from 0.00027
11/11 [==============================] - 0s 10ms/step - loss: 4.1301e-04 - val_loss: 3.6417e-04
Epoch 74/256
 8/11 [====================>.........] - ETA: 0s - loss: 4.0097e-04
Epoch 00074: val_los

In [7]:
nrmse_list

[0.18095653887288063,
 0.21610342220545428,
 0.17703420936560738,
 0.18483345317900285,
 0.1787689365523665,
 0.15920666683517615,
 0.20056110633635524,
 0.16172132638385844,
 0.2002872456749483,
 0.1979806797189743,
 0.15824703445197744]

In [9]:
path = os.path.join(args.root, 'results', 'automl', 'result.csv')
result = pd.read_csv(path)

nrmse_list = []
y_test = result['y_test'].values
for i in range(11):
    y_pred = result['%dth model' % (i + 1)]
    zero_indices = np.where(y_test == 0)
    y_test = np.delete(y_test, zero_indices)
    y_pred = np.delete(y_pred, zero_indices)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    max_min = np.max(y_test) - np.min(y_test)
    nrmse = rmse / max_min
    nrmse_list.append(nrmse)

    print('rmse:', rmse)
    print('nrmse:', nrmse)
    print('max:', np.max(y_test))
    print

nrmse_list

rmse: 59.83868125335656
nrmse: 0.19491427118357185
rmse: 50.28353375453916
nrmse: 0.16379001222976924
rmse: 54.31076318767862
nrmse: 0.17690802341263392
rmse: 75.84732396671404
nrmse: 0.24705968718799362
rmse: 91.20473039111073
nrmse: 0.29708381234889486
rmse: 84.34914097802114
nrmse: 0.27475290220853793
rmse: 86.58299018003912
nrmse: 0.2820292839740688
rmse: 58.074629348498625
nrmse: 0.18916817377361117
rmse: 49.78102095662546
nrmse: 0.16215316272516436
rmse: 47.579036831690686
nrmse: 0.15498057599899245
rmse: 77.93427881672993
nrmse: 0.2538575857222473


[0.19491427118357185,
 0.16379001222976924,
 0.17690802341263392,
 0.24705968718799362,
 0.29708381234889486,
 0.27475290220853793,
 0.2820292839740688,
 0.18916817377361117,
 0.16215316272516436,
 0.15498057599899245,
 0.2538575857222473]